In [ ]:
import plotly.express as px
import pandas as pd

In [ ]:
def lire_annotations_csv(nom_fichier: str) -> pd.DataFrame:
    df = pd.read_csv(nom_fichier, delimiter=';').fillna(False)
    for col in df.columns:
        if col in {"token", "Token"}:
            continue

        df[col] = df[col].astype(bool)
    return df


In [ ]:
annotations_spacy = lire_annotations_csv("data_eval_NER//Noailles-spacy.csv")
annotations_spacy


In [ ]:
def calculer_pr_re_f1(vp: int, fp: int, fn: int) -> tuple[float, float, float]:
    precision = vp / (vp + fp) if (vp + fp) > 0 else 0
    rappel = vp / (vp + fn) if (vp + fn) > 0 else 0
    f1 = 0
    if vp > 0:
        f1 = 2 * precision * rappel / (precision + rappel)
    return precision, rappel, f1


In [ ]:
def vp_vn_fp_fn(gold_ent: bool, trouve_ent: bool) -> str:
    if gold_ent and trouve_ent:
        return "VP"
    elif not gold_ent and trouve_ent:
        return "FP"
    elif gold_ent and not trouve_ent:
        return "FN"
    else:
        # On ajoute le vrai négatif même si on ne l'utilisera pas pour le calcul des métriques autant être exhaustif
        return "VN"

In [ ]:
def comparer_csv(gold_csv: str, system_csv: str, do_print:bool = False) -> dict:
    gold = lire_annotations_csv(gold_csv)
    system = lire_annotations_csv(system_csv)

    # Dans un DataFrame polars, on distingue height et width, respectivement, le nombre de lignes et le nombre de colonnes
    if gold.shape[0] != system.shape[0]:
        raise ValueError("Les fichiers n'ont pas le même nombre de lignes.")

    # Du coup, on a aussi rajouté les vrais négatifs ici
    dic_global = {"VP": 0, "FP": 0, "FN": 0, "VN": 0}

    dic_cat = {
        "PER": {"VP": 0, "FP": 0, "FN": 0, "VN": 0},
        "LOC": {"VP": 0, "FP": 0, "FN": 0, "VN": 0},
        "MISC": {"VP": 0, "FP": 0, "FN": 0, "VN": 0},
        "ORG": {"VP": 0, "FP": 0, "FN": 0, "VN": 0}
    }

    for i in range(gold.shape[0]):
        annot_g = gold.iloc[i]
        annot_s = system.iloc[i]

        gold_cat = {key: value for key, value in annot_g.items() if key not in {"token", "Token"}}
        gold_global = any(gold_cat.values())
        system_cat = {key: value for key, value in annot_s.items() if key not in {"token", "Token"}}
        system_global = any(system_cat.values())

        resultat_global = vp_vn_fp_fn(gold_global, system_global)
        dic_global[resultat_global] += 1

        for cat in dic_cat.keys():
            resultat_courant = vp_vn_fp_fn(gold_cat[cat], system_cat[cat])
            dic_cat[cat][resultat_courant] += 1

    if do_print:
        print("Global:", dic_global)
        for cat, compteurs in dic_cat.items():
            print(f"Catégorie {cat}: {compteurs}")

    return {
        "global": dic_global,
        **dic_cat,
    }


In [ ]:
def calculer_et_afficher_metrics(dic_compteurs: dict, do_print: bool = False) -> dict:
    res = {}
    for categorie, compteurs in dic_compteurs.items():
        vp = compteurs["VP"]
        fp = compteurs["FP"]
        fn = compteurs["FN"]
        precision, rappel, f1 = calculer_pr_re_f1(vp, fp, fn)

        if do_print:
            print(f"Catégorie {categorie}:")
            print(f"  Précision: {precision:.4f}")
            print(f"  Rappel: {rappel:.4f}")
            print(f"  F1-score: {f1:.4f}")

        res[categorie] = {
            "precision": precision,
            "rappel": rappel,
            "f1": f1
        }
    return res


In [ ]:
resultat = comparer_csv("data_eval_NER//Noailles1.csv", "data_eval_NER//Noailles-spacy.csv", do_print=True)

calculer_et_afficher_metrics(resultat, do_print=True)


# Visualisation des résultats avec Plotly

In [ ]:
def tracer_courbes(dic_compteurs: dict, show: bool = True):
    res = # TODO: calculez les métriques en réutilisant la fonction précédente

    df = (
        pd
        .DataFrame
        .from_dict(res, orient='index')
        .reset_index() # contrairement à polars, pandas a toujours un index (nommé index par défaut)
        .rename(columns={'index': 'Catégorie'})
    )

    # Dans l'exemple avec polars, on avait décidé de faire un melt pour réorganiser les données
    # Cependant, on peut aussi utiliser directement plotly express sans faire de melt
    # Au lieu de faire notre transformation, on peut passer plusieurs colonnes à y dans px.bar
    # et plotly express se chargera de faire le melt en interne

    fig = px.bar(
        df,
        x="", # TODO: la colonne de l'axe "x"
        y=[], # TODO: les colonnes de l'axe "y" à tracer
        barmode='group',
        title='Métriques de performance par catégorie',
        labels={
            'value': "", # TODO: nommez les axes
            'variable': "", # TODO: nommez les axes
        }
    )
    if show:
        # TODO: Affichez la figure
    return fig



Vous pouvez désormais tester la fonction de traçage